# Setting Up the PostgreSQL Schema with pgvector in Python

In [8]:
import os
from dotenv import load_dotenv 
import json
import psycopg2
from psycopg2.extras import Json

In [2]:
# Check connection to database

# Load environment variables from .env file
load_dotenv() 
# Get PostgreSQL connection details from environment variables
username = os.getenv('PG_ADMIN_USERNAME')  # Get the server admin login name stored in .env file
password = os.getenv('PG_ADMIN_PASSWORD')  # Get the server password stored in .env file
host = os.getenv('PG_SERVER_NAME')         # Get the server name stored in .env file
port = os.getenv('POSTGRES_PORT')          # Get the Standard PostgreSQL port stored in .env file
database = os.getenv('PG_DATABASE')        # Get the database name stored in .env file

# The PostgreSQL connection string 
POSTGRESQL_CONNECTION = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Connect to the PostgreSQL database
def connect_to_db():
    connection = psycopg2.connect(POSTGRESQL_CONNECTION)
    return connection

# Test the connection
try:
    conn = connect_to_db()
    cursor = conn.cursor()
    print("Nice! Connection successful!")
    cursor.close()
    conn.close()
except Exception as e:
    print(f"Error: {e}")

Nice! Connection successful!


In [11]:
# Check 3: Check existing tables in the database

def list_tables():
    try:
        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        tables = cursor.fetchall()
        
        print("Tables:", [table[0] for table in tables] if tables else "No tables found.")
        
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error listing tables: {e}")

list_tables()

Tables: ['talk']


In [4]:
# Setup our table. It will be called talk

# Create a cursor object to execute SQL commands
conn = connect_to_db()
cursor = conn.cursor()

# SQL query to create the table with metadata field containing topic, question_title, and source
create_table_query = """
    CREATE EXTENSION IF NOT EXISTS vector;

    CREATE TABLE IF NOT EXISTS talk (
        question_id VARCHAR(50) PRIMARY KEY,
        question_full TEXT,
        answers JSONB,
        vector VECTOR(1536),  -- Will be using text-embedding-ada-002 (OpenAI) which produces 1536-dimension vectors
        metadata JSONB  -- This will contain topic, question_title, and source
    );
"""

try:
    # Execute the query
    cursor.execute(create_table_query)
    
    # Commit changes
    conn.commit()
    print("Table created successfully!")
    
except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close cursor and connection
    cursor.close()
    conn.close()

list_tables()

Table created successfully!


In [21]:
# Drop table

conn = connect_to_db()
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS talk;")
conn.commit()

print("Table 'talk' has been dropped.")

# Close the cursor and connection
cursor.close()
conn.close()

Table 'talk' has been dropped.


In [22]:
list_tables()

Tables: No tables found.
